In [1]:
import json
from google.cloud import secretmanager

In [6]:
# Initialize the Secret Manager client
client = secretmanager.SecretManagerServiceClient()

project_id = "dev-posigen"
secret_id = "dev-cx-voiceai"
version_id="1"

secret_path = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

In [7]:
# Step 1: Retrieve the current secret value
response = client.access_secret_version(request={"name": secret_path})
current_secret = json.loads(response.payload.data.decode("utf-8"))  # Convert JSON string to dict

In [10]:
# Step 2: Read the new .p8 key file
with open("secrets/snowflakegcp_rsa_key.p8", "r") as f:
    private_key_data = f.read()
    
# Step 3: Append the private key data to the JSON
current_secret["private_key"] = private_key_data  # Add a new key or update existing

# Step 4: Convert back to JSON and upload a new version
updated_secret_value = json.dumps(current_secret)

In [12]:
# Step 5: Add a new version of the updated secret
client.add_secret_version(
    request={"parent": f"projects/{project_id}/secrets/{secret_id}", 
             "payload": {"data": updated_secret_value.encode("utf-8")}}
)

print("✅ Successfully updated the secret with the .p8 private key!")

✅ Successfully updated the secret with the .p8 private key!
